In [ ]:
#| default_exp utils

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from __future__ import annotations

import torch

from diffdrr.detector import Detector

In [ ]:
#| export
def reshape_subsampled_drr(
    img: torch.Tensor, 
    detector: Detector, 
    batch_size: int,
):
    n_points = detector.height * detector.width
    drr = torch.zeros(batch_size, n_points).to(img)
    drr[:, detector.subsamples[-1]] = img
    drr = drr.view(batch_size, 1, detector.height, detector.width)
    return drr

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()